In [5]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from TOV_Solver import WaveNetTOV, evaluate_model
import matplotlib.pyplot as plt
import joblib
from preprocessing import _resample_group, tov_load_and_preprocess
from EOS_Solver import chi2_loss
import copy

In [6]:
test = pd.read_csv("data/sample_eos.csv")
test["rho"] = test["rho"]
test[['ID', "p", 'rho']]
unique_ids_test = test["ID"].unique().tolist()

max = test.rho.max()
min = test.rho.min()

In [7]:
print(f"Test set density range: {min} to {max}")

Test set density range: 0.06 to 1.28


In [8]:
test.query(f"ID == {unique_ids_test[0]}")

,ID,model,weight,e,p,rho
0,8293.0,RMFNL,1.217783e-06,47.410754,0.031089,0.06
1,8293.0,RMFNL,1.217783e-06,56.876734,0.087275,0.07
2,8293.0,RMFNL,1.217783e-06,66.353718,0.141353,0.08
3,8293.0,RMFNL,1.217783e-06,75.839753,0.201318,0.09
4,8293.0,RMFNL,1.217783e-06,85.334896,0.276774,0.10
...,...,...,...,...,...,...
12263710,8293.0,GDFMX,8.762089e-07,1632.385290,782.559500,1.24
12263711,8293.0,GDFMX,8.762089e-07,1651.929646,796.901111,1.25
12263712,8293.0,GDFMX,8.762089e-07,1671.587792,811.360000,1.26
12263713,8293.0,GDFMX,8.762089e-07,1691.359710,825.936003,1.27


In [9]:
sample_size = 64
N_samples = 1000
eos_data = pd.read_csv(
    "data/Monmoy/eos_cs2.csv",
    nrows = N_samples * 287 # for 40000 samples -> 11480000 rows ; here we have 287 rows per sample 
)

In [12]:
rho_grid = np.round(np.arange(0.06, 1.28 + 1e-12, 0.01), 2)

ID_COL = "ID"
RHO_COL = "rho"
P_COL = "p"

def interp_one_id(g: pd.DataFrame) -> pd.DataFrame:
    # group key value (the ID)
    id_value = g.name

    g = g[[RHO_COL, P_COL]].copy()

    g = g.sort_values(RHO_COL)
    g = g.groupby(RHO_COL, as_index=False)[P_COL].mean()

    x = g[RHO_COL].to_numpy()
    y = g[P_COL].to_numpy()

    if len(x) < 2:
        return pd.DataFrame({ID_COL: id_value, RHO_COL: rho_grid, P_COL: np.nan})

    y_grid = np.interp(rho_grid, x, y, left=np.nan, right=np.nan)

    return pd.DataFrame({ID_COL: id_value, RHO_COL: rho_grid, P_COL: y_grid})

eos_interp = (
    eos_data
    .dropna(subset=[ID_COL, RHO_COL, P_COL])
    .groupby(ID_COL, group_keys=False)
    .apply(interp_one_id)
    .reset_index(drop=True)
)

print(eos_interp.head())


   ID   rho         p
0   7  0.06  0.317310
1   7  0.07  0.417404
2   7  0.08  0.526109
3   7  0.09  0.679272
4   7  0.10  0.878662


In [14]:
eos_data = eos_interp.copy()
eos_data['model'] = "RMF"
unique_ids = eos_data["ID"].unique().tolist()

In [15]:
mr_data = pd.read_csv("data/Monmoy/mr_cs2.csv")
mr_data = mr_data[mr_data["ID"].isin(unique_ids)]
mr_data

,ID,ec,M,R,Lambda
0,7,289.411765,0.812132,12.897442,8729.581483
1,7,313.613445,0.994052,12.847777,3211.489512
2,7,337.815126,1.172509,12.863732,1380.417947
3,7,362.016807,1.337366,12.895802,683.882018
4,7,386.218487,1.487452,12.923066,376.615989
...,...,...,...,...,...
34227,6686,894.453782,2.563226,12.432985,5.885787
34228,6686,918.655462,2.564164,12.404504,5.707931
34229,6686,942.857143,2.564527,12.385730,5.598142
34230,6686,967.058824,2.564620,12.373439,5.529952


In [16]:
mr_data.ID.nunique()

1000

In [17]:
# input_df = eos_data.copy()
# Np = 64

# # --- Normalize density (ρ / ρ_sat) ---
# input_df["rho"] = input_df["rho"] / 0.16

# # --- Resample EoS and MR curves ---
# input_interp = input_df.groupby(["ID", "model"], group_keys=False).apply(
#     _resample_group, column="rho", Np=Np, include_groups=True
# )


# # --- Prepare arrays ---
# N = input_df.ID.nunique()
# X = np.zeros((N, Np, 2), dtype=np.float32)

# # --- Build dataset ---
# for i in range(N):
#     ID = unique_ids[i]
#     eos_grp = input_interp.loc[(input_interp["ID"] == ID)]

#     # log10(p)
#     p_vals = eos_grp["p"].to_numpy()

#     X[i, :, 1] = np.log10(np.clip(p_vals, 1e-30, None))

#     rho_phys = input_df["rho"].loc[(input_df["ID"] == ID) ].to_numpy(dtype=np.float32)
#     rho_min, rho_max = float(rho_phys.min()), float(rho_phys.max())

#     rho_phys_targets = np.linspace(rho_min, rho_max, Np, dtype=np.float32)
#     rho_scaled = 0.1 * (rho_phys_targets - rho_min) / (rho_max - rho_min)

#     X[i, :, 0] = rho_scaled

#     if i % 1000 == 0:
#         print(f"Processed {i}/{N}")

# # --- Scale X globally ---
# X_max = np.load("scalers/X_MAX.npy")
# X_scaled = X.copy()
# X_scaled[:, :, 1] = X[:, :, 1] / X_max

# np.save("data/Monmoy/X_scaled.npy", X_scaled)

# X_scaled = np.load("data/Monmoy/X_scaled.npy")

In [18]:
X_scaled = np.load("data/Monmoy/X_scaled.npy")

In [21]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("torch cuda version:", torch.version.cuda)


torch: 2.10.0+cpu
cuda available: False
torch cuda version: None


In [19]:
device = torch.device("cuda")

ckpt = torch.load("models/tov_solver.pt", map_location=device)

# 3️⃣ If it’s a dict:
model = WaveNetTOV().to(device)
model.load_state_dict(ckpt)
model.eval()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# Prepare tensors
X_test = torch.tensor(X_scaled[:,:,1:2], dtype=torch.float32).to(device)

# Evaluation
with torch.no_grad():
    pred = model(X_test)
pred = pred.cpu().numpy()

In [ ]:
M_MAX = np.load("scalers/M_MAX.npy")
R_MAX = 16.0
R_MIN = np.load("scalers/R_MIN.npy")

mass_pred_unscaled = pred[:, :, 0]  * M_MAX
radius_pred_unscaled = pred[:, :, 1] * (R_MAX - R_MIN) + R_MIN

In [ ]:
# Plot a single sample (set `idx` to choose which sample)
idx = 0
fig, ax = plt.subplots(figsize=(6, 6))

# True M-R relation (black)
ax.scatter(
    mr_data.query(f"ID == @unique_ids[{idx}]").R.values,
    mr_data.query(f"ID == @unique_ids[{idx}]").M.values,
    color='black',
    label='True'
)

# Predicted M-R relation (red dashed)
ax.plot(
    radius_pred_unscaled[idx],
    mass_pred_unscaled[idx],
    "--",
    color='red',
    label='Predicted'
)

ax.set_title(f'Sample {idx + 1}')
ax.set_xlabel('Radius')
ax.set_ylabel('Mass')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
n_samples_to_plot = 10
fig, axes = plt.subplots(2, 5, figsize=(18, 7))  # 2 rows × 5 cols
axes = axes.flatten()

for i in range(n_samples_to_plot):
    ax = axes[i]

    # True M-R relation (black)
    ax.scatter(
        mr_data.query(f"ID == @unique_ids[{i}]").R.values,
        mr_data.query(f"ID == @unique_ids[{i}]").M.values,
        color='black',
        label='True'
    )

    # Predicted M-R relation (red dashed)
    ax.plot(
        radius_pred_unscaled[i],
        mass_pred_unscaled[i],
        "--",
        color='red',
        label='Predicted'
    )

    ax.set_title(f'Sample {i + 1}')
    ax.set_xlabel('Radius')
    ax.set_ylabel('Mass')
    ax.grid(True, alpha=0.3)

    # Only show legend once (cleaner look)
    if i == 0:
        ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
dM = np.ones_like(radius_pred_unscaled[0], dtype=np.float32)
dR = np.ones_like(radius_pred_unscaled[0], dtype=np.float32)


In [ ]:
total_loss = 0.0
mass_chi2_term = 0.0
radius_chi2_term = 0.0

for k in range(N_samples):  # k indexes IDs
    mr_pred_t = torch.tensor(pred[k], dtype=torch.float32).unsqueeze(0)
    M_seq, R_seq = mr_pred_t[0,:,0], mr_pred_t[0,:,1]

    mr_data_ = mr_data.query(f"ID == @unique_ids[{k}]")
    m_obs_t = torch.tensor(mr_data_.M.values / M_MAX, dtype=torch.float32)
    r_obs_t = torch.tensor((mr_data_.R.values - R_MIN) / (R_MAX - R_MIN), dtype=torch.float32)
    dM_t = torch.tensor(dM, dtype=torch.float32)
    dR_t = torch.tensor(dR, dtype=torch.float32)

    for j in range(len(m_obs_t)):  # j indexes observations for that ID
        distances = ((M_seq - m_obs_t[j])/(dM_t[j] + 1e-6))**2 + \
                    ((R_seq - r_obs_t[j])/(dR_t[j] + 1e-6))**2

        min_idx = torch.argmin(distances)

        m_term = ((M_seq[min_idx] - m_obs_t[j])/(dM_t[j] + 1e-6))**2
        r_term = ((R_seq[min_idx] - r_obs_t[j])/(dR_t[j] + 1e-6))**2

        total_loss += m_term + r_term
        mass_chi2_term += m_term
        radius_chi2_term += r_term

In [ ]:

print("Mass chi2 term:", mass_chi2_term.item()/N_samples)
print("Radius chi2 term:", radius_chi2_term.item()/N_samples)
print(f"Total Chi² Loss over test set: {total_loss/(N_samples)}")